In [26]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

class FS():
    def __init__(self,X, y, labeled):
        self.X = X
        self.nDim = X.shape[1]
        self.labels = np.unique(y)
        self.nClass = len(np.unique(y))
        self.y = y
        self.labeled = labeled
        self.fs = self.attribute_weight()

    def attribute_weight(self):
        # ------------Initialize some things------------
        total_Mean = np.mean(self.X[self.labeled], axis=0)
        class_Num = np.zeros(self.nClass)
        class_Mean = np.zeros((self.nClass, self.nDim))
        class_std = np.zeros((self.nClass, self.nDim))
        fisher_score_list = np.zeros(self.nDim)

        # --------calculate intermediate components---------
        labeled_y = self.y[self.labeled]
        labeled_idx = np.array(self.labeled)
        for i, lab in enumerate(self.labels):
            ids = np.where(labeled_y == lab)[0]
            lab_ids = labeled_idx[ids]  # 类别为lab的已标记样本索引
            class_Num[i] = len(lab_ids)
            class_Mean[i] = np.mean(self.X[lab_ids], axis=0)
            class_std[i] = np.std(self.X[lab_ids], axis=0)

        # --utlize the intermediate components to compute the FS------
        for j in range(self.nDim):
            Sb_j = 0.0
            Sw_j = 0.0
            for i in range(self.nClass):
                Sb_j += class_Num[i] * (class_Mean[i,j] - total_Mean[j]) ** 2
                Sw_j += class_Num[i] * class_std[i,j] ** 2
            fisher_score_list[j] = Sb_j / Sw_j
        print("FS::",fisher_score_list)
        fisher_score_list = np.where(np.isnan(fisher_score_list), 0, fisher_score_list)
        print("FS::",fisher_score_list)

        # -----softmax based normalize FS range to [0,1]------
        Norm_fs = np.exp(fisher_score_list)
        Norm_fs_sum = sum(Norm_fs)
        Norm_fs = Norm_fs / Norm_fs_sum

        return Norm_fs

df=pd.read_csv('data/room.csv')
feature_columns = [ col for col in df.columns if col not in ['id']]
for f in  feature_columns:
    labelencoder=LabelEncoder()
    df[f] = labelencoder.fit_transform(df[f])
feature_columns = [ col for col in df.columns if col not in ['id','classes']]
Y = df["classes"]
X = df[feature_columns]

# Compute the feature importance using Fisher score
fs_model = FS(X=X.values, y=Y.values, labeled=df['id'].values-1)
feature_importance = fs_model.fs

# Print the feature importance values
print("Feature importance:", feature_importance)

top_indexes = np.argsort(feature_importance)[::-1][:9]
top_features = [feature_columns[index] for index in top_indexes]
print(top_features)

FS:: [0.00279841 0.03288812 0.00098448 0.33647214 0.00885266 0.01697276
 0.13811383 0.41151044 0.39180497 0.01049004 0.16844636 0.12616668
 0.0981072  0.02432315 0.02203232        nan 0.02151688 0.04815806
 0.20437896 0.03050878 0.09801663 0.04941098]
FS:: [0.00279841 0.03288812 0.00098448 0.33647214 0.00885266 0.01697276
 0.13811383 0.41151044 0.39180497 0.01049004 0.16844636 0.12616668
 0.0981072  0.02432315 0.02203232 0.         0.02151688 0.04815806
 0.20437896 0.03050878 0.09801663 0.04941098]
Feature importance: [0.04082904 0.04207625 0.04075505 0.05700092 0.04107698 0.04141189
 0.04674508 0.06144274 0.06024384 0.0411443  0.0481847  0.04618994
 0.04491189 0.0417174  0.04162195 0.04071495 0.0416005  0.04272368
 0.04994759 0.04197625 0.04490782 0.04277724]
['gill-size', 'gill-color', 'bruises', 'ring-type', 'stalk-root', 'gill-spacing', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'population']


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars


In [27]:
from sklearn import svm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold

df = pd.read_csv('data/room.csv')
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id', 'classes']]
Y = df["classes"]
X = df[top_features]
def SVM(trainX, trainY, testX, testY):
    scaler = StandardScaler()
    scaler.fit(trainX)
    trainX3 = scaler.transform(trainX)
    testX3 = scaler.transform(testX)
    clf = svm.SVC(C=10, gamma=0.8, max_iter=500)
    clf.fit(trainX3, trainY)
    test_result = clf.predict(testX3)
    precision = np.sum(test_result == testY) / testY.shape[0]
    return precision

        # Use stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = []
for train_index, test_index in skf.split(X, Y):
    trainX, testX = X.iloc[train_index], X.iloc[test_index]
    trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
    score = SVM(trainX, trainY, testX, testY)
    scores.append(score)
avg_score = np.mean(scores)
print(avg_score)

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv

0.991136337471749


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


In [28]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

df = pd.read_csv('data/room.csv')
feature_columns = [ col for col in df.columns if col not in ['id']]
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id','classes']]
X = df[top_features]
Y = df['classes']
dt = DecisionTreeClassifier(max_depth=5)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

accuracy_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    dt.fit(X_train, y_train)

    y_pred = dt.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(mean_accuracy)

0.9665156417574027


In [29]:
from sklearn.neighbors import KNeighborsClassifier

k = 5
knn = KNeighborsClassifier(n_neighbors=k)
df = pd.read_csv('data/room.csv')
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id','classes']]
Y=df["classes"]
X=df[top_features]
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracy_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(mean_accuracy)

0.9940904877029066
